In [1]:
def evaluate_2(res, frac):
    assert len(res) == 60
    score = 1/3 + frac/3
    rcount = 0
    pcount = 0
    scount = 0
    
    allcount = 1
    if res[0] == "R":
        pcount += 1
    if res[0] == "S":
        rcount += 1
    if res[0] == "P":
        scount += 1

    for x in res[1:]:
        if x == "R":
            score += (scount/allcount) + (rcount/allcount)*frac
            pcount += 1
        if x == "S":
            score += (pcount/allcount) + (scount/allcount)*frac
            rcount += 1
        if x == "P":
            score += (rcount/allcount) + (pcount/allcount)*frac
            scount += 1
        allcount += 1
    
    return score

In [2]:
def gen(arr):
    rps = "RSP"
    srr = ""
    for i,x in enumerate(arr):
        srr += rps[i%3]*x
    return srr

In [3]:
r10 = evaluate_2("RPS"*20, 1)
r10

41.64047105353967

In [4]:
r05 = evaluate_2("RPS"*20, 0.5)
r05

30.720394650024275

In [5]:
r05 = evaluate_2(gen((1, 5, 15, 23, 16)), 0.5)
r05

33.043061433829976

In [6]:
r01 = evaluate_2("RPS"*20, 0.1)
r01

21.98433352721195

In [7]:
r01 = evaluate_2(gen((1, 4, 15, 25, 15)), 0.1)
r01

29.146994921167078

In [8]:
r00 = evaluate_2("RPS"*20, 0)
r00

19.80031824650886

In [9]:
r00 = evaluate_2(gen((0, 1, 9, 27, 23)), 0.0)
r00

28.466022908980722

In [10]:
500*(r10 + r05 + r01 + r00)/4

16537.06878968968

In [11]:
r10, r05, r01, r00

(41.64047105353967, 33.043061433829976, 29.146994921167078, 28.466022908980722)

In [ ]:
best = 0
best_idx = 0
for i in range(60):
    for j in range(60-i):
        for k in range(60-i-j):
            for l in range(60-i-j-k):
                m = 60-i-j-k-l
                r00 = evaluate_2("R"*i+"S"*j+"P"*k+"R"*l+"S"*m, 0.5)
                if r00 > best:
                    best_idx = (i,j,k,l,m)
                    best = r00

In [ ]:
best_idx, best